# 简介

交叉验证（Cross Validation）是常用的一种用来评估模型效果的方法。

当样本分布发生变化时，交叉验证无法准确评估模型在测试集上的效果，这导致模型在测试集上的效果远低于训练集。

通过本文，你将通过一个kaggle的比赛实例了解到，样本分布变化如何影响建模，如何通过对抗验证辨别样本的分布变化，以及有哪些应对方法。

# 什么是「样本分布变化」？

在真实的业务场景中，我们经常会遇到「样本分布变化」的问题。

主要体现在训练集和测试集的分布存在的差异。比如，在化妆品或者医美市场，男性的比例越来越多。基于过去的数据构建的模型，渐渐不适用于现在。

# 为什么「样本分布变化」的时候，交叉验证不适用？

当我们要做一个模型，来预测人们在超市的消费习惯。

我们的训练样本主要是18岁-25岁的年轻人构成，而测试样本主要是70岁以上的老人组成。这时样本分布就发生了变化。

![样本分布变化](./images/Change_in_Distribution.png)

这种情况下，使用交叉验证，其实无法准确评估模型的效果。原因是，交叉验证的验证集和测试集不够相似。

交叉验证中，每一折的验证集都是从训练集随机抽取的。随机抽取的验证集的分布和整体的训练集是相同的，也就意味着每一折的验证集都和测试集的分布存在较大的差异。

所以在样本分布变化时，通过交叉验证的方式构建的模型，在测试集上的表现相较于训练集通常会打折扣。稍后我们会通过一个实例来确认这一点。

# 什么是对抗验证（Adversarial Validation）？

[对抗验证（Adversarial Validation）](http://fastml.com/adversarial-validation-part-one/)，并不像交叉验证是一种评估模型效果的方法，而是一种用来确认训练集和测试集的分布是否变化的方法。

它的本质是，构造一个分类模型，来预测样本是训练集或测试集的概率。

如果这个模型的效果不错（通常来说AUC在0.7以上），那么可以说明我们的训练集和测试集存在较大的差异。

如下图，仍然以「预测人们在超市的消费习惯」为例。因为训练集主要是18岁-25岁的年轻人，测试集主要是70岁以上的老人，那么通过「年龄」，我们就能够比较好的区分出训练集和测试集。

<img src="./images/Adversarial_Validation.png" width="500" height="500" />

具体步骤如下：

* 定义新的Y（因变量）：样本是train还是test。训练集中的样本统一标记为0，测试集则标记为1。
* 将 Train 和 Test 合成一个数据集
* 构造一个模型，拟合新定义的Y。
* 观察模型效果：如果模型的AUC超过0.7，说明了 Train 和 Test 的分布存在较大的差异。

# 分布变化时，优于交叉验证的方法

主要是三种方法：

* 人工划分验证集
* 和测试集最相似的样本作为验证集
* 有权重的交叉验证

## 人工划分验证集

人工划分验证集，需要我们对数据有充分的了解。

因为我知道这次比赛的数据是根据时间划分的，所以我的验证集同样可以根据时间划分。

如果我们不清楚训练集和测试集如何划分，可以采用后面两种方法。

## 用和测试集分布最相似的样本，作为验证集

如果对数据没有充分了解，如何找到训练集中，和测试集分布最相似的样本呢？

这就会用到我们做对抗验证时，模型预测样本是测试集的概率。概率越高，则说明和测试集越相似。

## 有权重的交叉验证

不仅可以用对抗验证中，模型预测样本是测试集的概率来划分验证集，也可以将这个概率作为样本的权重。

概率越高，和测试集就越相似，权重就越高。

这样，我们就可以做有权重的交叉验证。

# 实例

## 数据

![Microsoft Malware Prediction](./images/Microsoft_Malware_Competition.png)


这里用到的数据来自Kaggle上的[微软恶意软件比赛](https://www.kaggle.com/c/microsoft-malware-prediction/overview)。

每一个样本代表着一台电脑。这次比赛的目标是：预测电脑受到恶意软件攻击的概率。

因为这次比赛的 Train 和 Test 是根据时间划分的，所以Train 和 Test 的分布非常不同，很具有代表性。

如果需要数据，可以从[Kaggle](https://www.kaggle.com/c/microsoft-malware-prediction/data)下载。

## Import

In [1]:
import pandas as pd
from tqdm import tqdm
import lightgbm as lgb
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold

# Memory management
import gc
gc.enable()

# Plot
import seaborn as sns
import matplotlib.pyplot as plt
plt.style.use('ggplot')

# Suppress warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# dtypes = {
#     'MachineIdentifier':                                    'category',
#     'ProductName':                                          'category',
#     'EngineVersion':                                        'category',
#     'AppVersion':                                           'category',
#     'AvSigVersion':                                         'category',
#     'IsBeta':                                               'int8',
#     'RtpStateBitfield':                                     'float16',
#     'IsSxsPassiveMode':                                     'int8',
#     'DefaultBrowsersIdentifier':                            'float32',
#     'AVProductStatesIdentifier':                            'float32',
#     'AVProductsInstalled':                                  'float16',
#     'AVProductsEnabled':                                    'float16',
#     'HasTpm':                                               'int8',
#     'CountryIdentifier':                                    'int16',
#     'CityIdentifier':                                       'float32',
#     'OrganizationIdentifier':                               'float16',
#     'GeoNameIdentifier':                                    'float16',
#     'LocaleEnglishNameIdentifier':                          'int16',
#     'Platform':                                             'category',
#     'Processor':                                            'category',
#     'OsVer':                                                'category',
#     'OsBuild':                                              'int16',
#     'OsSuite':                                              'int16',
#     'OsPlatformSubRelease':                                 'category',
#     'OsBuildLab':                                           'category',
#     'SkuEdition':                                           'category',
#     'IsProtected':                                          'float16',
#     'AutoSampleOptIn':                                      'int8',
#     'PuaMode':                                              'category',
#     'SMode':                                                'float16',
#     'IeVerIdentifier':                                      'float16',
#     'SmartScreen':                                          'category',
#     'Firewall':                                             'float16',
#     'UacLuaenable':                                         'float64',  # was 'float32'
#     'Census_MDC2FormFactor':                                'category',
#     'Census_DeviceFamily':                                  'category',
#     'Census_OEMNameIdentifier':                             'float32',  # was 'float16'
#     'Census_OEMModelIdentifier':                            'float32',
#     'Census_ProcessorCoreCount':                            'float16',
#     'Census_ProcessorManufacturerIdentifier':               'float16',
#     'Census_ProcessorModelIdentifier':                      'float32',  # was 'float16'
#     'Census_ProcessorClass':                                'category',
#     'Census_PrimaryDiskTotalCapacity':                      'float64',  # was 'float32'
#     'Census_PrimaryDiskTypeName':                           'category',
#     'Census_SystemVolumeTotalCapacity':                     'float64',  # was 'float32'
#     'Census_HasOpticalDiskDrive':                           'int8',
#     'Census_TotalPhysicalRAM':                              'float32',
#     'Census_ChassisTypeName':                               'category',
#     'Census_InternalPrimaryDiagonalDisplaySizeInInches':    'float32',  # was 'float16'
#     'Census_InternalPrimaryDisplayResolutionHorizontal':    'float32',  # was 'float16'
#     'Census_InternalPrimaryDisplayResolutionVertical':      'float32',  # was 'float16'
#     'Census_PowerPlatformRoleName':                         'category',
#     'Census_InternalBatteryType':                           'category',
#     'Census_InternalBatteryNumberOfCharges':                'float64',  # was 'float32'
#     'Census_OSVersion':                                     'category',
#     'Census_OSArchitecture':                                'category',
#     'Census_OSBranch':                                      'category',
#     'Census_OSBuildNumber':                                 'int16',
#     'Census_OSBuildRevision':                               'int32',
#     'Census_OSEdition':                                     'category',
#     'Census_OSSkuName':                                     'category',
#     'Census_OSInstallTypeName':                             'category',
#     'Census_OSInstallLanguageIdentifier':                   'float16',
#     'Census_OSUILocaleIdentifier':                          'int16',
#     'Census_OSWUAutoUpdateOptionsName':                     'category',
#     'Census_IsPortableOperatingSystem':                     'int8',
#     'Census_GenuineStateName':                              'category',
#     'Census_ActivationChannel':                             'category',
#     'Census_IsFlightingInternal':                           'float16',
#     'Census_IsFlightsDisabled':                             'float16',
#     'Census_FlightRing':                                    'category',
#     'Census_ThresholdOptIn':                                'float16',
#     'Census_FirmwareManufacturerIdentifier':                'float16',
#     'Census_FirmwareVersionIdentifier':                     'float32',
#     'Census_IsSecureBootEnabled':                           'int8',
#     'Census_IsWIMBootEnabled':                              'float16',
#     'Census_IsVirtualDevice':                               'float16',
#     'Census_IsTouchEnabled':                                'int8',
#     'Census_IsPenCapable':                                  'int8',
#     'Census_IsAlwaysOnAlwaysConnectedCapable':              'float16',
#     'Wdft_IsGamer':                                         'float16',
#     'Wdft_RegionIdentifier':                                'float16',
#     'HasDetections':                                        'int8'
# }

# df_all = pd.read_csv('./input/train.csv.zip', dtype=dtypes) 
## 对训练集随机抽取2%的样本
# df_all = df_all.sample(frac=0.02, random_state=123)
# df_all.to_csv('./input/train_sample.csv', index=False)

因为这次比赛的训练集有800万样本，测试集有700万样本。为了方便演示，这里我仅随机抽取训练集中2%的样本，而且不使用测试集的数据。我们稍后将从训练集中拆分一个数据集，作为我们的测试集。

不使用这次比赛原本的测试集可以节省很多时间。因为测试集有700万的样本，每做一次预测会消耗大量时间。

In [3]:
# 读取训练集中随机抽取的2%的样本
df_all = pd.read_csv('./input/train_sample.csv') 
df_all.head()

,MachineIdentifier,ProductName,EngineVersion,AppVersion,AvSigVersion,IsBeta,RtpStateBitfield,IsSxsPassiveMode,DefaultBrowsersIdentifier,AVProductStatesIdentifier,...,Census_FirmwareVersionIdentifier,Census_IsSecureBootEnabled,Census_IsWIMBootEnabled,Census_IsVirtualDevice,Census_IsTouchEnabled,Census_IsPenCapable,Census_IsAlwaysOnAlwaysConnectedCapable,Wdft_IsGamer,Wdft_RegionIdentifier,HasDetections
0,586d40804b950d0376575fdf10ee89ae,win8defender,1.1.15100.1,4.18.1806.18062,1.273.520.0,0,7.0,0,NaN,53447.0,...,27767.0,1,NaN,0.0,0,0,0.0,1.0,15.0,1
1,65fb3fae2d37f90e6b3174592f2490a8,win8defender,1.1.15200.1,4.18.1807.18075,1.275.453.0,0,7.0,0,NaN,7945.0,...,14353.0,0,NaN,0.0,0,0,0.0,0.0,10.0,0
2,c23aa37fb69e00afe2668ed150dee1ea,win8defender,1.1.15100.1,4.18.1807.18075,1.273.689.0,0,7.0,0,NaN,53447.0,...,8941.0,1,NaN,0.0,0,0,0.0,1.0,1.0,1
3,cba75d6c4d9b6533591e94b9cb8a5df5,win8defender,1.1.15200.1,4.12.16299.15,1.275.483.0,0,7.0,0,NaN,68585.0,...,46589.0,1,NaN,0.0,0,0,0.0,1.0,7.0,1
4,149746364c6b763662d03e1f263029fd,win8defender,1.1.15200.1,4.18.1807.18075,1.275.215.0,0,7.0,0,NaN,53447.0,...,52530.0,0,NaN,0.0,0,0,0.0,NaN,NaN,0


本次比赛的数据中提供了电脑的 Windows Defender（Windows系统自带的杀毒软件）版本号，所以我们可以通过该本版号发布的时间，粗略的推测采集该样本的时间。

这里AvSigVersionTimestamps就是各个Windows Defender版本对应的发布时间。

通过和该数据匹配，我们生成了一个新的字段 -- Date（日期）。这个字段稍后会起作用。

In [4]:
# 读取Windows Defender版本对应的发布时间
datedict = np.load('./input/AvSigVersionTimestamps.npy')
datedict = datedict[()]

# 生成新的变量Date
df_all['Date'] = df_all['AvSigVersion'].map(datedict)

# MachineIdentifier是每台电脑的唯一识别号，对于模型的预测没有任何帮助，所以剔除。
df_all.drop(['MachineIdentifier'], axis=1, inplace=True) 

## 数据清理

### 去掉无意义变量

这里无意义变量的定义是：变量的某个值（可以是空值）的占比大于99%。

比如，如果所有样本的「系统版本」都是Win7，那么「系统版本」这个变量就没有意义。

所以，如果一个变量，99%以上的样本，都是一个值，那么这个变量接近于无意义。

In [5]:
bad_cols = []
for col in df_all.columns:
    rate_train = df_all[col].value_counts(normalize=True, dropna=False).values[0]
    if rate_train > 0.99:
        bad_cols.append(col)

df_all = df_all.drop(bad_cols, axis=1)

print('Data Shape: ', df_all.shape)
print(bad_cols)

Data Shape:  (178430, 75)
['IsBeta', 'AutoSampleOptIn', 'PuaMode', 'UacLuaenable', 'Census_DeviceFamily', 'Census_ProcessorClass', 'Census_IsPortableOperatingSystem', 'Census_IsVirtualDevice']


### 定义数据类型

这里是通过EDA(Exploratory Data Analysis)的方式，人工判断的变量类型。

总共将变量分为
* 数值变量（true_numerical_columns）
* 一般的分类变量（categorical_columns）
* 类别非常多的分类变量（categorical_columns_high_car）：比如中国的城市（北京、上海、深圳、重庆等等等...）

如果你对这次比赛的细节感兴趣，可以再深入研究为什么这样判断。这里就不详细阐述原因了。

In [6]:
true_numerical_columns = [
    'Census_PrimaryDiskTotalCapacity', 'Census_SystemVolumeTotalCapacity',
    'Census_TotalPhysicalRAM', 'Census_InternalBatteryNumberOfCharges'
]

categorical_columns_high_car = [
    'Census_FirmwareVersionIdentifier', 'Census_OEMModelIdentifier',
    'AVProductStatesIdentifier', 'Census_FirmwareManufacturerIdentifier',
    'Census_InternalPrimaryDiagonalDisplaySizeInInches',
    'Census_InternalPrimaryDisplayResolutionHorizontal',
    'Census_InternalPrimaryDisplayResolutionVertical',
    'Census_OEMNameIdentifier', 'Census_ProcessorModelIdentifier',
    'CityIdentifier', 'DefaultBrowsersIdentifier', 'OsBuildLab'
]

categorical_columns = [
    c for c in df_all.columns
    if c not in (['HasDetections', 'Date'] + true_numerical_columns +
                 categorical_columns_high_car)
]
print(categorical_columns)

['ProductName', 'EngineVersion', 'AppVersion', 'AvSigVersion', 'RtpStateBitfield', 'IsSxsPassiveMode', 'AVProductsInstalled', 'AVProductsEnabled', 'HasTpm', 'CountryIdentifier', 'OrganizationIdentifier', 'GeoNameIdentifier', 'LocaleEnglishNameIdentifier', 'Platform', 'Processor', 'OsVer', 'OsBuild', 'OsSuite', 'OsPlatformSubRelease', 'SkuEdition', 'IsProtected', 'SMode', 'IeVerIdentifier', 'SmartScreen', 'Firewall', 'Census_MDC2FormFactor', 'Census_ProcessorCoreCount', 'Census_ProcessorManufacturerIdentifier', 'Census_PrimaryDiskTypeName', 'Census_HasOpticalDiskDrive', 'Census_ChassisTypeName', 'Census_PowerPlatformRoleName', 'Census_InternalBatteryType', 'Census_OSVersion', 'Census_OSArchitecture', 'Census_OSBranch', 'Census_OSBuildNumber', 'Census_OSBuildRevision', 'Census_OSEdition', 'Census_OSSkuName', 'Census_OSInstallTypeName', 'Census_OSInstallLanguageIdentifier', 'Census_OSUILocaleIdentifier', 'Census_OSWUAutoUpdateOptionsName', 'Census_GenuineStateName', 'Census_ActivationChan

### 编码 -- Label Encoding 

因为将使用的模型是[LightGBM](https://lightgbm.readthedocs.io/en/latest/)，所以我们需要对分类变量做编码。

这里用的方法是[Label Encoding](http://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.factorize.html)。

In [7]:
def factor_data(df, col):
    df_labeled, _ = df[col].factorize(sort=True)
    # MAKE SMALLEST LABEL 1, RESERVE 0
    df_labeled += 1
    # MAKE NAN LARGEST LABEL
    df_labeled = np.where(df_labeled==0, df_labeled.max()+1, df_labeled)
    df[col] = df_labeled

for col in tqdm(categorical_columns + categorical_columns_high_car):
    factor_data(df_all, col) 

100%|██████████| 69/69 [00:01<00:00, 40.89it/s]


## 构造测试集

像刚才提到的，因为没有使用测试集的数据，所以我们需要从训练集中拆分出一个数据集，作为我们的测试集，用于评价我们评估模型的方式是否有效。

因为训练集和测试集是根据时间划分的，所以我们从训练集拆分的测试集，同样也根据时间划分。

这是为了尽量模拟真实的测试集。

In [8]:
# 将样本根据时间排序
df_all = df_all.sort_values('Date').reset_index(drop=True) 
df_all.drop(['Date'], axis=1, inplace=True)

In [9]:
# 将前80%的样本作为训练集，后20%的样本作为测试集
df_test = df_all.iloc[int(0.8*len(df_all)):, ]
df_train = df_all.iloc[:int(0.8*len(df_all)), ]

## 对抗验证（Adversarial Validatiion）

In [10]:
# 定义新的Y
df_train['Is_Test'] = 0
df_test['Is_Test'] = 1

# 将 Train 和 Test 合成一个数据集。HasDetections是数据本来的Y，所以剔除。
df_adv = pd.concat([df_train, df_test])

adv_data = lgb.Dataset(
    data=df_adv.drop('Is_Test', axis=1), label=df_adv.loc[:, 'Is_Test'])

# 定义模型参数
params = {
    'boosting_type': 'gbdt',
    'colsample_bytree': 1,
    'learning_rate': 0.1,
    'max_depth': 5,
    'min_child_samples': 100,
    'min_child_weight': 1,
    'min_split_gain': 0.0,
    'num_leaves': 20,
    'objective': 'binary',
    'random_state': 50,
    'subsample': 1.0,
    'subsample_freq': 0,
    'metric': 'auc',
    'num_threads': 8
}

# 交叉验证
adv_cv_results = lgb.cv(
    params,
    adv_data,
    num_boost_round=10000,
    nfold=5,
    categorical_feature=categorical_columns,
    early_stopping_rounds=200,
    verbose_eval=True,
    seed=42)

print('交叉验证中最优的AUC为 {:.5f}，对应的标准差为{:.5f}.'.format(
    adv_cv_results['auc-mean'][-1], adv_cv_results['auc-stdv'][-1]))

print('模型最优的迭代次数为{}.'.format(len(adv_cv_results['auc-mean'])))

[1]	cv_agg's auc: 0.981538 + 0.000780946
[2]	cv_agg's auc: 0.991886 + 0.0011813
[3]	cv_agg's auc: 0.997781 + 0.00022218
[4]	cv_agg's auc: 0.998255 + 0.000357413
[5]	cv_agg's auc: 0.999215 + 0.000110628
[6]	cv_agg's auc: 0.999382 + 4.43679e-05
[7]	cv_agg's auc: 0.999477 + 3.37784e-05
[8]	cv_agg's auc: 0.999488 + 4.45144e-05
[9]	cv_agg's auc: 0.999513 + 4.41991e-05
[10]	cv_agg's auc: 0.999587 + 4.13675e-05
[11]	cv_agg's auc: 0.999624 + 5.00616e-05
[12]	cv_agg's auc: 0.999663 + 3.64739e-05
[13]	cv_agg's auc: 0.999683 + 3.87281e-05
[14]	cv_agg's auc: 0.999708 + 1.98319e-05
[15]	cv_agg's auc: 0.999731 + 7.15555e-06
[16]	cv_agg's auc: 0.999756 + 2.11395e-05
[17]	cv_agg's auc: 0.999771 + 1.06916e-05
[18]	cv_agg's auc: 0.999799 + 2.30115e-05
[19]	cv_agg's auc: 0.999816 + 1.94767e-05
[20]	cv_agg's auc: 0.999836 + 2.90241e-05
[21]	cv_agg's auc: 0.999859 + 3.2347e-05
[22]	cv_agg's auc: 0.999867 + 2.67817e-05
[23]	cv_agg's auc: 0.999874 + 3.19173e-05
[24]	cv_agg's auc: 0.999875 + 2.13535e-05
[25]	

[195]	cv_agg's auc: 0.999985 + 6.99561e-06
[196]	cv_agg's auc: 0.999985 + 7.02045e-06
[197]	cv_agg's auc: 0.999985 + 7.35668e-06
[198]	cv_agg's auc: 0.999984 + 7.90237e-06
[199]	cv_agg's auc: 0.999984 + 7.65815e-06
[200]	cv_agg's auc: 0.999984 + 7.71088e-06
[201]	cv_agg's auc: 0.999984 + 7.65929e-06
[202]	cv_agg's auc: 0.999984 + 7.44411e-06
[203]	cv_agg's auc: 0.999984 + 7.26212e-06
[204]	cv_agg's auc: 0.999985 + 6.71573e-06
[205]	cv_agg's auc: 0.999985 + 6.73701e-06
[206]	cv_agg's auc: 0.999985 + 6.7113e-06
[207]	cv_agg's auc: 0.999985 + 6.58684e-06
[208]	cv_agg's auc: 0.999985 + 6.76216e-06
[209]	cv_agg's auc: 0.999985 + 6.84919e-06
[210]	cv_agg's auc: 0.999984 + 6.64034e-06
[211]	cv_agg's auc: 0.999984 + 6.57349e-06
[212]	cv_agg's auc: 0.999984 + 6.43365e-06
[213]	cv_agg's auc: 0.999985 + 5.99357e-06
[214]	cv_agg's auc: 0.999985 + 6.07347e-06
[215]	cv_agg's auc: 0.999985 + 6.1671e-06
[216]	cv_agg's auc: 0.999985 + 6.09936e-06
[217]	cv_agg's auc: 0.999985 + 6.03495e-06
[218]	cv_agg'

通过对抗验证，我们发现模型的AUC达到了0.99。说明本次比赛的训练集和测试集的样本分布存在较大的差异。

然后，我们使用训练好的模型，对所有的样本进行预测，得到各个样本属于测试集的概率。这个之后会用到。

In [11]:
params['n_estimators'] = len(adv_cv_results['auc-mean'])

model_adv = lgb.LGBMClassifier(**params)
model_adv.fit(df_adv.drop('Is_Test', axis=1), df_adv.loc[:, 'Is_Test'])

preds_adv = model_adv.predict_proba(df_adv.drop('Is_Test', axis=1))[:, 1]

## 交叉验证（Cross Validation）

现在我们知道了训练集和测试集的分布存在很大的差异。那么接下来，我们采用交叉验证的方法，来评估模型的效果。

In [12]:
def run_cv(df_train, sample_weight=None):
    if sample_weight is not None:
        train_set = lgb.Dataset(
            df_train.drop('HasDetections', axis=1),
            label=df_train.loc[:, 'HasDetections'], weight=sample_weight)

    else:
        train_set = lgb.Dataset(
            df_train.drop('HasDetections', axis=1),
            label=df_train.loc[:, 'HasDetections'])

    # Perform cross validation with early stopping
    params.pop('n_estimators', None)
    
    N_FOLDS = 5
    cv_results = lgb.cv(
        params,
        train_set,
        num_boost_round=10000,
        nfold=N_FOLDS,
        categorical_feature=categorical_columns,
        early_stopping_rounds=200,
        verbose_eval=True,
        seed=42)

    print('交叉验证中最优的AUC为 {:.5f}，对应的标准差为{:.5f}.'.format(
        cv_results['auc-mean'][-1], cv_results['auc-stdv'][-1]))

    print('模型最优的迭代次数为{}.'.format(len(cv_results['auc-mean'])))

    params['n_estimators'] = len(cv_results['auc-mean'])

    model_cv = lgb.LGBMClassifier(**params)
    model_cv.fit(df_train.drop('HasDetections', axis=1),
                 df_train.loc[:, 'HasDetections'])

    # AUC
    preds_test_cv = model_cv.predict_proba(
        df_test.drop('HasDetections', axis=1))[:, 1]
    auc_test_cv = roc_auc_score(df_test.loc[:, 'HasDetections'], preds_test_cv)
    print('模型在测试集上的效果是{:.5f}。'.format(
        auc_test_cv))

    return model_cv

In [13]:
model_cv = run_cv(df_train)

[1]	cv_agg's auc: 0.673642 + 0.00265052
[2]	cv_agg's auc: 0.678893 + 0.00448619
[3]	cv_agg's auc: 0.681506 + 0.00347297
[4]	cv_agg's auc: 0.682945 + 0.00385881
[5]	cv_agg's auc: 0.684684 + 0.0041531
[6]	cv_agg's auc: 0.686289 + 0.00343128
[7]	cv_agg's auc: 0.687549 + 0.00316743
[8]	cv_agg's auc: 0.688257 + 0.0034225
[9]	cv_agg's auc: 0.689064 + 0.00333125
[10]	cv_agg's auc: 0.690081 + 0.00332507
[11]	cv_agg's auc: 0.690898 + 0.00355769
[12]	cv_agg's auc: 0.691641 + 0.00343357
[13]	cv_agg's auc: 0.692108 + 0.00359948
[14]	cv_agg's auc: 0.692707 + 0.00384981
[15]	cv_agg's auc: 0.693462 + 0.00350958
[16]	cv_agg's auc: 0.693609 + 0.00341991
[17]	cv_agg's auc: 0.69383 + 0.00328001
[18]	cv_agg's auc: 0.694183 + 0.00329412
[19]	cv_agg's auc: 0.694634 + 0.00340595
[20]	cv_agg's auc: 0.695012 + 0.00318532
[21]	cv_agg's auc: 0.69514 + 0.00324965
[22]	cv_agg's auc: 0.695565 + 0.00343536
[23]	cv_agg's auc: 0.696031 + 0.00331183
[24]	cv_agg's auc: 0.696424 + 0.00332546
[25]	cv_agg's auc: 0.696618 +

[201]	cv_agg's auc: 0.699909 + 0.00332299
[202]	cv_agg's auc: 0.699903 + 0.003358
[203]	cv_agg's auc: 0.699889 + 0.00332874
[204]	cv_agg's auc: 0.699886 + 0.00328193
[205]	cv_agg's auc: 0.699943 + 0.00320245
[206]	cv_agg's auc: 0.699879 + 0.00322782
[207]	cv_agg's auc: 0.699792 + 0.0032461
[208]	cv_agg's auc: 0.699732 + 0.00325164
[209]	cv_agg's auc: 0.69971 + 0.00319165
[210]	cv_agg's auc: 0.699629 + 0.00319212
[211]	cv_agg's auc: 0.699643 + 0.00320773
[212]	cv_agg's auc: 0.699651 + 0.00324131
[213]	cv_agg's auc: 0.699618 + 0.00330046
[214]	cv_agg's auc: 0.699601 + 0.00327513
[215]	cv_agg's auc: 0.699595 + 0.00326767
[216]	cv_agg's auc: 0.69956 + 0.00324504
[217]	cv_agg's auc: 0.699553 + 0.0032811
[218]	cv_agg's auc: 0.699539 + 0.00330446
[219]	cv_agg's auc: 0.699516 + 0.00330204
[220]	cv_agg's auc: 0.699502 + 0.00329667
[221]	cv_agg's auc: 0.699485 + 0.00328499
[222]	cv_agg's auc: 0.699456 + 0.00328292
[223]	cv_agg's auc: 0.69946 + 0.00325112
[224]	cv_agg's auc: 0.699471 + 0.00325479

使用交叉验证的方式来评估模型效果：

* 交叉验证AUC：0.70144
* 测试集上AUC：0.66980
* 差值：0.03

交叉验证和测试集上的AUC差值较大，说明交叉验证的方式不太能准确评估模型在测试集上的效果。 

我们再来来试一下其他方法，对比看看。

## 在变量分布变化的情况下，除了交叉验证，还有哪些更优的方法？

### 人工划分验证集

In [14]:
def run_lgb(df_train, df_validation):
    dtrain = lgb.Dataset(
        data=df_train.drop('HasDetections', axis=1),
        label=df_train.loc[:, 'HasDetections'],
        free_raw_data=False,
        silent=True)

    dvalid = lgb.Dataset(
        data=df_validation.drop('HasDetections', axis=1),
        label=df_validation.loc[:, 'HasDetections'],
        free_raw_data=False,
        silent=True)

    params.pop('n_estimators', None)

    clf = lgb.train(
        params=params,
        train_set=dtrain,
        num_boost_round=10000,
        valid_sets=[dtrain, dvalid],
        early_stopping_rounds=200,
        verbose_eval=True,
        categorical_feature=categorical_columns)

    params['n_estimators'] = clf.num_trees()

    model = lgb.LGBMClassifier(**params)
    model.fit(
        df_train.drop('HasDetections', axis=1),
        df_train.loc[:, 'HasDetections'])

    # AUC
    preds_test = model.predict_proba(
        df_test.drop('HasDetections', axis=1))[:, 1]
    auc_test = roc_auc_score(df_test.loc[:, 'HasDetections'], preds_test)
    print('模型在测试集上的效果是{:.5f}。'.format(
        roc_auc_score(df_test.loc[:, 'HasDetections'], preds_test)))
    return model

In [15]:
# 之前已经用Date进行了排序，所以提取出后20%的样本作为验证集。
df_validation_1 = df_train.iloc[int(0.8 * len(df_train)):, ]
df_train_1 = df_train.iloc[:int(0.8 * len(df_train)), ]

In [16]:
model_1 = run_lgb(df_train_1, df_validation_1)

[1]	training's auc: 0.688654	valid_1's auc: 0.642369
Training until validation scores don't improve for 200 rounds.
[2]	training's auc: 0.69429	valid_1's auc: 0.649962
[3]	training's auc: 0.696418	valid_1's auc: 0.651748
[4]	training's auc: 0.698366	valid_1's auc: 0.653697
[5]	training's auc: 0.699429	valid_1's auc: 0.654109
[6]	training's auc: 0.70092	valid_1's auc: 0.654634
[7]	training's auc: 0.704122	valid_1's auc: 0.655645
[8]	training's auc: 0.706059	valid_1's auc: 0.657076
[9]	training's auc: 0.706789	valid_1's auc: 0.657017
[10]	training's auc: 0.708772	valid_1's auc: 0.659064
[11]	training's auc: 0.709966	valid_1's auc: 0.659128
[12]	training's auc: 0.710595	valid_1's auc: 0.65896
[13]	training's auc: 0.712962	valid_1's auc: 0.660827
[14]	training's auc: 0.714985	valid_1's auc: 0.66104
[15]	training's auc: 0.716574	valid_1's auc: 0.661911
[16]	training's auc: 0.717889	valid_1's auc: 0.662541
[17]	training's auc: 0.718961	valid_1's auc: 0.663312
[18]	training's auc: 0.719463	va

[154]	training's auc: 0.792496	valid_1's auc: 0.675089
[155]	training's auc: 0.792854	valid_1's auc: 0.675041
[156]	training's auc: 0.793164	valid_1's auc: 0.67512
[157]	training's auc: 0.793717	valid_1's auc: 0.67515
[158]	training's auc: 0.794199	valid_1's auc: 0.675206
[159]	training's auc: 0.794418	valid_1's auc: 0.675215
[160]	training's auc: 0.794797	valid_1's auc: 0.675297
[161]	training's auc: 0.795373	valid_1's auc: 0.675284
[162]	training's auc: 0.795846	valid_1's auc: 0.67528
[163]	training's auc: 0.796273	valid_1's auc: 0.675307
[164]	training's auc: 0.796541	valid_1's auc: 0.675257
[165]	training's auc: 0.796964	valid_1's auc: 0.675325
[166]	training's auc: 0.797298	valid_1's auc: 0.675346
[167]	training's auc: 0.797701	valid_1's auc: 0.675336
[168]	training's auc: 0.798072	valid_1's auc: 0.675393
[169]	training's auc: 0.79877	valid_1's auc: 0.675341
[170]	training's auc: 0.799108	valid_1's auc: 0.675308
[171]	training's auc: 0.799536	valid_1's auc: 0.675373
[172]	training

[304]	training's auc: 0.836358	valid_1's auc: 0.676398
[305]	training's auc: 0.83645	valid_1's auc: 0.676455
[306]	training's auc: 0.83658	valid_1's auc: 0.676541
[307]	training's auc: 0.836857	valid_1's auc: 0.676503
[308]	training's auc: 0.837038	valid_1's auc: 0.676554
[309]	training's auc: 0.837406	valid_1's auc: 0.676536
[310]	training's auc: 0.837637	valid_1's auc: 0.676515
[311]	training's auc: 0.837871	valid_1's auc: 0.676428
[312]	training's auc: 0.837993	valid_1's auc: 0.676398
[313]	training's auc: 0.838232	valid_1's auc: 0.67638
[314]	training's auc: 0.838359	valid_1's auc: 0.67633
[315]	training's auc: 0.838496	valid_1's auc: 0.676291
[316]	training's auc: 0.838676	valid_1's auc: 0.676284
[317]	training's auc: 0.83903	valid_1's auc: 0.676386
[318]	training's auc: 0.839311	valid_1's auc: 0.676441
[319]	training's auc: 0.839522	valid_1's auc: 0.676442
[320]	training's auc: 0.839726	valid_1's auc: 0.67645
[321]	training's auc: 0.840028	valid_1's auc: 0.676362
[322]	training's

[458]	training's auc: 0.867299	valid_1's auc: 0.676295
[459]	training's auc: 0.867513	valid_1's auc: 0.676271
[460]	training's auc: 0.867669	valid_1's auc: 0.676243
[461]	training's auc: 0.867871	valid_1's auc: 0.676233
[462]	training's auc: 0.868058	valid_1's auc: 0.67625
[463]	training's auc: 0.868158	valid_1's auc: 0.676284
[464]	training's auc: 0.868357	valid_1's auc: 0.67629
[465]	training's auc: 0.868459	valid_1's auc: 0.676286
[466]	training's auc: 0.868663	valid_1's auc: 0.676333
[467]	training's auc: 0.868967	valid_1's auc: 0.676324
[468]	training's auc: 0.869075	valid_1's auc: 0.676315
[469]	training's auc: 0.869303	valid_1's auc: 0.676338
[470]	training's auc: 0.869479	valid_1's auc: 0.676325
[471]	training's auc: 0.869716	valid_1's auc: 0.676374
[472]	training's auc: 0.870054	valid_1's auc: 0.676351
[473]	training's auc: 0.87014	valid_1's auc: 0.676348
[474]	training's auc: 0.870416	valid_1's auc: 0.676358
[475]	training's auc: 0.870602	valid_1's auc: 0.676362
[476]	trainin

使用人工划分验证集的方式来评估模型效果：

* 验证集AUC：0.676554
* 测试集上AUC：0.67228
* 差值：0.004

验证集和测试集上的AUC非常接近，说明人工划分的验证集，更能够评估模型在测试集上的效果。

这是因为人工划分的验证集，比起交叉验证的方式，和测试集更相似。

### 和测试集最相似的样本作为验证集

In [17]:
# 提取出训练集上，样本是测试集的概率
df_train_copy = df_train.copy()
df_train_copy['is_test_prob'] = preds_adv[:len(df_train)]

# 根据概率排序
df_train_copy = df_train_copy.sort_values('is_test_prob').reset_index(drop=True)

# 将概率最大的20%作为验证集
df_validation_2 = df_train_copy.iloc[int(0.8 * len(df_train)):, ]
df_train_2 = df_train_copy.iloc[:int(0.8 * len(df_train)), ]

df_validation_2.drop('is_test_prob', axis=1, inplace=True)
df_train_2.drop('is_test_prob', axis=1, inplace=True)

In [18]:
model_2 = run_lgb(df_train_2, df_validation_2)

[1]	training's auc: 0.688213	valid_1's auc: 0.642285
Training until validation scores don't improve for 200 rounds.
[2]	training's auc: 0.692815	valid_1's auc: 0.64517
[3]	training's auc: 0.696436	valid_1's auc: 0.648787
[4]	training's auc: 0.698626	valid_1's auc: 0.651753
[5]	training's auc: 0.700533	valid_1's auc: 0.653136
[6]	training's auc: 0.702502	valid_1's auc: 0.655382
[7]	training's auc: 0.703482	valid_1's auc: 0.656062
[8]	training's auc: 0.705527	valid_1's auc: 0.656927
[9]	training's auc: 0.706809	valid_1's auc: 0.65691
[10]	training's auc: 0.708477	valid_1's auc: 0.657349
[11]	training's auc: 0.709249	valid_1's auc: 0.657707
[12]	training's auc: 0.710815	valid_1's auc: 0.658642
[13]	training's auc: 0.713306	valid_1's auc: 0.658696
[14]	training's auc: 0.714219	valid_1's auc: 0.659128
[15]	training's auc: 0.715036	valid_1's auc: 0.658691
[16]	training's auc: 0.715994	valid_1's auc: 0.659692
[17]	training's auc: 0.717335	valid_1's auc: 0.659841
[18]	training's auc: 0.718214	

[155]	training's auc: 0.791454	valid_1's auc: 0.662756
[156]	training's auc: 0.791628	valid_1's auc: 0.662718
[157]	training's auc: 0.79222	valid_1's auc: 0.662656
[158]	training's auc: 0.792484	valid_1's auc: 0.662597
[159]	training's auc: 0.792547	valid_1's auc: 0.662653
[160]	training's auc: 0.792786	valid_1's auc: 0.662752
[161]	training's auc: 0.79313	valid_1's auc: 0.662767
[162]	training's auc: 0.793479	valid_1's auc: 0.662701
[163]	training's auc: 0.793723	valid_1's auc: 0.662779
[164]	training's auc: 0.793951	valid_1's auc: 0.662746
[165]	training's auc: 0.794366	valid_1's auc: 0.662699
[166]	training's auc: 0.794605	valid_1's auc: 0.66273
[167]	training's auc: 0.794897	valid_1's auc: 0.662607
[168]	training's auc: 0.795341	valid_1's auc: 0.66278
[169]	training's auc: 0.795454	valid_1's auc: 0.662776
[170]	training's auc: 0.795825	valid_1's auc: 0.66264
[171]	training's auc: 0.796271	valid_1's auc: 0.662607
[172]	training's auc: 0.796526	valid_1's auc: 0.66259
[173]	training's

使用这种方式来评估模型效果：

* 验证集AUC：0.665711
* 测试集上AUC：0.66196
* 差值：0.003

差值同样远小于交叉验证的方式。

### 有权重的交叉验证 

In [19]:
model_cv_wight = run_cv(df_train, sample_weight=preds_adv[:len(df_train)])

[1]	cv_agg's auc: 0.651778 + 0.011999
[2]	cv_agg's auc: 0.660672 + 0.0155175
[3]	cv_agg's auc: 0.658344 + 0.0140189
[4]	cv_agg's auc: 0.65738 + 0.01787
[5]	cv_agg's auc: 0.657711 + 0.0179211
[6]	cv_agg's auc: 0.659267 + 0.0165729
[7]	cv_agg's auc: 0.657827 + 0.017638
[8]	cv_agg's auc: 0.657777 + 0.0181964
[9]	cv_agg's auc: 0.658422 + 0.0179104
[10]	cv_agg's auc: 0.658812 + 0.017411
[11]	cv_agg's auc: 0.657857 + 0.0176432
[12]	cv_agg's auc: 0.657417 + 0.0169003
[13]	cv_agg's auc: 0.655824 + 0.0178147
[14]	cv_agg's auc: 0.655964 + 0.0178522
[15]	cv_agg's auc: 0.65785 + 0.0187058
[16]	cv_agg's auc: 0.657498 + 0.0192713
[17]	cv_agg's auc: 0.657857 + 0.0194838
[18]	cv_agg's auc: 0.659321 + 0.0190793
[19]	cv_agg's auc: 0.659281 + 0.0190464
[20]	cv_agg's auc: 0.658585 + 0.0181283
[21]	cv_agg's auc: 0.657659 + 0.017343
[22]	cv_agg's auc: 0.657752 + 0.0179614
[23]	cv_agg's auc: 0.657745 + 0.0175434
[24]	cv_agg's auc: 0.6585 + 0.0175791
[25]	cv_agg's auc: 0.659771 + 0.0176137
[26]	cv_agg's auc: 

[205]	cv_agg's auc: 0.662923 + 0.00980105
[206]	cv_agg's auc: 0.66259 + 0.00966143
[207]	cv_agg's auc: 0.662717 + 0.00952947
[208]	cv_agg's auc: 0.662515 + 0.00954792
[209]	cv_agg's auc: 0.662059 + 0.00951722
[210]	cv_agg's auc: 0.662222 + 0.00975936
[211]	cv_agg's auc: 0.66226 + 0.00956029
[212]	cv_agg's auc: 0.662343 + 0.00935288
[213]	cv_agg's auc: 0.662179 + 0.00939192
[214]	cv_agg's auc: 0.66247 + 0.00913708
[215]	cv_agg's auc: 0.661948 + 0.00901557
[216]	cv_agg's auc: 0.662004 + 0.00888755
[217]	cv_agg's auc: 0.661982 + 0.00871169
[218]	cv_agg's auc: 0.662215 + 0.00878428
[219]	cv_agg's auc: 0.662359 + 0.00868966
[220]	cv_agg's auc: 0.662152 + 0.00872557
[221]	cv_agg's auc: 0.661934 + 0.00863083
[222]	cv_agg's auc: 0.662087 + 0.00871303
[223]	cv_agg's auc: 0.66207 + 0.00841599
[224]	cv_agg's auc: 0.662374 + 0.00840024
[225]	cv_agg's auc: 0.662394 + 0.00847774
[226]	cv_agg's auc: 0.662422 + 0.00868587
[227]	cv_agg's auc: 0.662675 + 0.00848627
[228]	cv_agg's auc: 0.66231 + 0.008368

使用有权重的交叉验证来评估模型效果：

* 有权重的交叉验证AUC：0.66511
* 测试集上AUC：0.66295
* 差值：0.002

差值同样小于交叉验证的方式。

## 对比各种方法效果

分别使用上述提到的总共4种方法，我们来对比一下四种方法的效果，如下表：

![对比表格](./images/Validation_Chart.png)

使用交叉验证时，验证集AUC和测试集AUC的差值是最大的，远高于其他方式。说明在样本分布发生变化时，交叉验证不能够准确评估模型在测试集上的效果。

## 为什么评价方式是差值，而不是测试集AUC？

有人可能会提到，哪种方法在测试集上的AUC最高，哪种方法就更好，不是吗？

需要注意的是，本文讨论的不是“提升”模型效果的方法，而是“评估”模型效果的方法。

具体来说，虽然目前看来，比如交叉验证在测试集上的AUC，略高于有权重的交叉验证。

但是，当前的模型只是一个很基础的模型（Baseline Model），没有做任何的变量筛选，特征工程，以及模型调参。

由于所有的优化模型的决定，都将基于验证集，而交叉验证无法准确评估模型在测试集上的效果，这将导致很多优化模型的决定是错误的。

只有在有一个可靠的验证集的情况下，提升模型在验证集上效果的方法，我们才有信心认为，它也可以提升在测试集上的表现。

另外，从本次比赛的结果，我们也可以发现，最终排名很好的参赛者，都没有使用交叉验证。

# 结论

在样本分布发生变化时，交叉验证不能够准确评估模型在测试集上的效果。

这里建议采用其他方式：

* 人工划分验证集
* 和测试集最相似的样本作为验证集
* 有权重的交叉验证

如果你有任何疑问或者建议，欢迎通过“机器学习小站”公众号留言，或者qiuyan.liu918@gmail.com联系我。